# Wywoływanie funkcji R z poziomu Pythona

Siłą rzeczy nie wszystkie przydatne funkcje (w szczególności nie wszystkie techniki statystyczne) są zaimplementowane w najpopularniejszych bibliotekach w Pythonie. Dobrą informacją jest to, że za pomocą pakietu `rpy2` możemy z łatwością transformować struktury danych `pandas` do standardowych struktur danych w R. Korzystanie z `pandas` omówimy dopiero w piątym module kursu, ale tutaj omówimy prosty przykład.

Bardziej rozbudowana dokumentacja znajduje się tutaj: https://rpy2.readthedocs.io/en/version_2.8.x/ i porusza wszystkie kwestie niezbędne do integracji R z Pythonem. Używałem tego pakietu wiele razy i często trzeba trochę pogłówkować i poczytać dokumentacje, ale co do za zasady działa on bardzo dobrze.

In [2]:
import pandas as pd
from pandas import DataFrame

Stwórzmy przykładowe dane.

In [3]:
data = DataFrame({
    'participant' : [1,2,3,4,5,6,7,8,9,10],
    'score' : [43,45,67,43,56,22,45,24,56,83],
    'sex' : ['K','M','K','M','K','K','K','M','M','K'],
    'condition' : ['E', 'C', 'E', 'C', 'E', 'C', 'E', 'C', 'E','C']
})
data.head()

,participant,score,sex,condition
0,1,43,K,E
1,2,45,M,C
2,3,67,K,E
3,4,43,M,C
4,5,56,K,E


Aby wykorzystać R w Pythonie musimy zaimportować kilka rzeczy:
+ `robjects` służy nam jako interfejs do obiektów R
+ `pandas2ri` służy (po wywołaniu `activate` do automatycznej konwersji obiektów `pandas` do obiektów R

In [4]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri, Formula
pandas2ri.activate()

Zróbmy psotą analizę wariancji. Najpierw stwórzmy model za pomocą R-owskiej funkcji `lm` i potem przekażmy go do funkcji `anova`.

In [26]:
result = robjects.r.lm('score ~ condition + sex', data = data)
print(robjects.r.anova(result))

Analysis of Variance Table

Response: score
          Df  Sum Sq Mean Sq F value Pr(>F)
condition  1  250.00  250.00  0.6577 0.4441
sex        1  121.68  121.68  0.3201 0.5892
Residuals  7 2660.72  380.10               



Jeśli chcemy tabelę z wynikami przekonwertować znów do obiektu `pandas` nic prostszego - wystarczy posłużyć się funkcją `ri2py` z modułu `pandas2ri`

In [27]:
anova_results = robjects.r.anova(result)
anova_results_py = pandas2ri.ri2py(anova_results)
anova_results_py.index = anova_results.rownames
anova_results_py

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
condition,1,250.00,250.000000,0.657717,0.444063
sex,1,121.68,121.680000,0.320124,0.589202
Residuals,7,2660.72,380.102857,NaN,NaN
